In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np

import tensorflow as tf

from tensorflow.keras.datasets import mnist

In [6]:
x=np.random.randn(2,4)
x

array([[ 0.92389455, -0.29853302, -1.15131816, -0.19887781],
       [ 1.30652212,  0.25715754,  0.39607276,  0.11390258]])

In [8]:
x.sum(axis=1)

array([-0.72483444,  2.073655  ])

### 加载数据，处理数据

In [101]:
(X_train,y_train),(X_test,y_test) = mnist.load_data()

# 归一化
X_train = (X_train/255.0).astype(np.float32)
X_test = (X_test/255.0).astype(np.float32)

# reshape,二维的数据，矩阵运算了
X_train = X_train.reshape(-1,784)
X_test = X_test.reshape(-1,784)

# one-hot 概率表示类别
y_train = tf.one_hot(y_train,depth=10)
y_test = tf.one_hot(y_test,depth=10)

# prefetch fetch取出来的意思，预先取出来，更快，加快计算速度
# 训练数据
data_train = tf.data.Dataset.from_tensor_slices((X_train,y_train)).repeat(50).shuffle(1000).batch(512).prefetch(1)
# 测试数据，验证模型好坏！每训练50次，取出一批测试数据，进行测试准确率
data_test = tf.data.Dataset.from_tensor_slices((X_test,y_test)).repeat(10).shuffle(1000).batch(512).prefetch(1)

### 构建模型

In [102]:
# 系数，3层隐含层
# 数据 X_train 样本，特征 784
# 第一层隐含层，多少个神经元：256个神经元，相当于线性方程，矩阵运算批量的计算
w1 = tf.Variable(tf.random.normal(shape = [784,256],stddev = 0.01))
b1 = tf.Variable(tf.random.normal(shape = [256],stddev = 0.01))

# 第二层隐含层，给256个神经元
w2 = tf.Variable(tf.random.normal(shape = [256,256],stddev = 0.01))
b2 = tf.Variable(tf.random.normal(shape = [256],stddev = 0.01))

#第三层，使用第二层数据的输出为数据
# 第三层给，512个神经元
w3 = tf.Variable(tf.random.normal(shape = [256,512],stddev = 0.01))
b3 = tf.Variable(tf.random.normal(shape = [512],stddev = 0.01))

# 输出层，输出几个类别，10分类问题，确定了，输出的类别是10个

out_w = tf.Variable(tf.random.normal(shape = [512,10],stddev = 0.01))
out_b = tf.Variable(tf.random.normal(shape = [10],stddev = 0.01))

def dnn(X):#深度神经网络
#     第一层的连接：矩阵运算
    d1 = tf.matmul(X,w1) + b1
    d1 = tf.nn.relu(d1) # 激活函数
    
#     第二层的连接，矩阵运算
    d2 = tf.nn.relu(tf.matmul(d1,w2) + b2)
    
#     第三层的连接，矩阵预算
    d3 = tf.nn.relu(tf.matmul(d2,w3) + b3)
    
#     输出层连接，矩阵运算，softmax概率问题
    y_pred = tf.nn.softmax(tf.matmul(d3,out_w) + out_b)
    return y_pred

# 损失函数，样本平均cross_entropy交叉熵
def cross_entropy(y_true,y_pred):
    y_pred = tf.clip_by_value(y_pred,1e-9,1.0)
    loss = tf.reduce_mean(tf.reduce_sum(y_true*tf.math.log(1/y_pred),axis = -1))#数据是二维的
    return loss

# 计算准确率
def accuracy(y_pred,y_true):#y_和y_true,都是tensor
    y_pred = y_pred.numpy().argmax(axis = -1)
    y_true = y_true.numpy().argmax(axis = -1)
    acc = (y_pred == y_true).mean()
    return acc #返回准确率
# 优化算法，梯度下降
# 学习率，太小，学的太慢！
sgd = tf.optimizers.SGD(learning_rate=0.2)

### 定义运行方法

In [103]:
def run_optimizer(X_train,y_train):
    with tf.GradientTape() as g:
        y_pred = dnn(X_train)
        loss = cross_entropy(y_train,y_pred)
    gradients = g.gradient(loss,[w1,b1,w2,b2,w3,b3,out_w,out_b])
    sgd.apply_gradients(zip(gradients,[w1,b1,w2,b2,w3,b3,out_w,out_b]))

### for循环运行代码

In [104]:
for i,(X_train,y_train) in enumerate(data_train.take(1000),1):
    run_optimizer(X_train,y_train)
    if i%10 == 0:
        #计算准确率
        for (X_test,y_test) in data_test.take(1):
            y_pred = dnn(X_test)
            acc = accuracy(y_pred,y_test)
            print('执行次数：%d。准确率是：%0.4f'%(i,acc))

执行次数：10。准确率是：0.1348
执行次数：20。准确率是：0.1191
执行次数：30。准确率是：0.1133
执行次数：40。准确率是：0.1270
执行次数：50。准确率是：0.1152
执行次数：60。准确率是：0.1270
执行次数：70。准确率是：0.1016
执行次数：80。准确率是：0.1328
执行次数：90。准确率是：0.1211
执行次数：100。准确率是：0.1191
执行次数：110。准确率是：0.1406
执行次数：120。准确率是：0.1289
执行次数：130。准确率是：0.1309
执行次数：140。准确率是：0.1348
执行次数：150。准确率是：0.1367
执行次数：160。准确率是：0.1309
执行次数：170。准确率是：0.1191
执行次数：180。准确率是：0.1113
执行次数：190。准确率是：0.1250
执行次数：200。准确率是：0.1191
执行次数：210。准确率是：0.1309
执行次数：220。准确率是：0.1348
执行次数：230。准确率是：0.1191
执行次数：240。准确率是：0.1074
执行次数：250。准确率是：0.1035
执行次数：260。准确率是：0.1328
执行次数：270。准确率是：0.1250
执行次数：280。准确率是：0.1035
执行次数：290。准确率是：0.1348
执行次数：300。准确率是：0.1211
执行次数：310。准确率是：0.1367
执行次数：320。准确率是：0.1426
执行次数：330。准确率是：0.1309
执行次数：340。准确率是：0.1426
执行次数：350。准确率是：0.1973
执行次数：360。准确率是：0.2012
执行次数：370。准确率是：0.2129
执行次数：380。准确率是：0.2285
执行次数：390。准确率是：0.2461
执行次数：400。准确率是：0.3496
执行次数：410。准确率是：0.4570
执行次数：420。准确率是：0.3867
执行次数：430。准确率是：0.4336
执行次数：440。准确率是：0.5566
执行次数：450。准确率是：0.5293
执行次数：460。准确率是：0.5938
执行次数：470。准确率是：0.5664
执行次数：480。准确率是：0.6406
执